In [17]:
//%cflags:-fopenmp -lm
#include <stdio.h>
#include "omp.h"

void main()
{
#pragma omp parallel num_threads(8) //inicio de region paralela
{
    int ID = omp_get_thread_num();
    printf(" hello( %d) ", ID);
    printf(" world( %d) \n", ID);
} //fin de region paralela
}

 hello( 0)  world( 0) 
 hello( 7)  hello( 4)  world( 7) 
 hello( 5)  world( 4) 
 world( 5) 
 hello( 6)  world( 6) 
 hello( 2)  world( 2) 
 hello( 3)  world( 3) 
 hello( 1)  world( 1) 


# Cálculo de pi
Forma básica con hilos OMP

In [54]:
//%cflags:-fopenmp -lm -D_DEFAULT_SOURCE
//gcc pi_omp.c -o pi_omp -fopenmp
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <sys/time.h>


#define ITERATIONS 2e09
#define THREADS 16



int calculatePi(double *piTotal, int ID)
{   int start, end;   

    start = (ITERATIONS/omp_get_num_threads()) * ID;
    end = (ITERATIONS/omp_get_num_threads()) * (ID + 1);
    int i = start;

    do{
        *(piTotal + ID) = *(piTotal + ID) + (double)(4.0 / ((i*2)+1));
        i++;
        *(piTotal + ID) = *(piTotal + ID) - (double)(4.0 / ((i*2)+1));
        i++;
    }while(i < end);

    return 0;
}


int main()
{
    int i, threads = THREADS;
    
    double pi[threads];
    
    struct timeval tval_before, tval_after, tval_result;

    gettimeofday(&tval_before, NULL);
    
    for(i = 0; i < THREADS; i++)
        pi[i] = 0;
        
    #pragma omp parallel num_threads(threads)
    {
        int ID = omp_get_thread_num();
        calculatePi(pi, ID);
    }
    
    for(i = 1; i < THREADS; i++)
    {
        pi[0] = pi[0] + pi[i];
    }
    gettimeofday(&tval_after, NULL);

    timersub(&tval_after, &tval_before, &tval_result);

    printf("Time elapsed: %ld.%06ld\n", (long int)tval_result.tv_sec, (long int)tval_result.tv_usec);
    
    printf("\npi: %2.10f   \n", pi[0]);
}

Time elapsed: 5.634394

pi: 3.1415926585   


## False sharing

In [55]:
//%cflags:-fopenmp -lm -D_DEFAULT_SOURCE
//gcc pi_omp.c -o pi_omp -fopenmp
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <sys/time.h>


#define ITERATIONS 2e09
#define THREADS 16
#define PAD 8


int calculatePi(double *piTotal, int ID)
{   int start, end;   

    start = (ITERATIONS/omp_get_num_threads()) * ID;
    end = (ITERATIONS/omp_get_num_threads()) * (ID + 1);
    int i = start;

    do{
        *(piTotal + (ID*PAD)) = *(piTotal + (ID*PAD)) + (double)(4.0 / ((i*2)+1));
        i++;
        *(piTotal + (ID*PAD)) = *(piTotal + (ID*PAD)) - (double)(4.0 / ((i*2)+1));
        i++;
    }while(i < end);

    return 0;
}


int main()
{
    int i, threads = THREADS;
    
    double pi[threads*PAD];
    
    struct timeval tval_before, tval_after, tval_result;

    gettimeofday(&tval_before, NULL);
    
    for(i = 0; i < THREADS; i++)
        pi[i*PAD] = 0;
        
    #pragma omp parallel num_threads(threads)
    {
        int ID = omp_get_thread_num();
        calculatePi(pi, ID);
    }
    
    for(i = 1; i < THREADS; i++)
    {
        *pi = *pi + *(pi + (i*PAD));
    }
    gettimeofday(&tval_after, NULL);

    timersub(&tval_after, &tval_before, &tval_result);

    printf("Time elapsed: %ld.%06ld\n", (long int)tval_result.tv_sec, (long int)tval_result.tv_usec);
    
    printf("\npi: %2.10f   \n", pi[0]);
}


Time elapsed: 2.855940

pi: 3.1415926585   


## atomic sync

In [3]:
//%cflags:-fopenmp -lm -D_DEFAULT_SOURCE
//gcc pi_omp.c -o pi_omp -fopenmp

#include <stdio.h>
#include <stdlib.h>
#include "omp.h"

#define ITERATIONS 2e09
#define THREADS 8

int calculatePi(double *piTotal, int ID)
{   int start, end;   

    start = (ITERATIONS/omp_get_num_threads()) * ID;
    end = (ITERATIONS/omp_get_num_threads()) * (ID + 1);
    int i = start;

    do{
        *(piTotal + ID) = *(piTotal + ID) + (double)(4.0 / ((i*2)+1));
        i++;
        *(piTotal + ID) = *(piTotal + ID) - (double)(4.0 / ((i*2)+1));
        i++;
    }while(i < end);

    return 0;
}


int main()
{
    int i, threads = THREADS;
    double pi[threads], piTotal;
    
    for(i = 0; i < THREADS; i++)
        pi[i] = 0;
    piTotal = 0;

    #pragma omp parallel num_threads(threads)
    {
        int ID = omp_get_thread_num();
        calculatePi(pi, ID);
       #pragma omp atomic
        piTotal = piTotal + pi[ID];
    }
      
    printf("\npi: %2.10f \n", piTotal);
}



pi: 3.1415926585 


## schedule

In [28]:
//%cflags:-fopenmp -lm -D_DEFAULT_SOURCE
//gcc omp_schedule.c -o omp_schedule -fopenmp

#include <stdio.h>
#include <omp.h>
#include <unistd.h>

#define NUM_THREADS 4
#define STATIC_CHUNK 5
#define DYNAMIC_CHUNK 5
#define NUM_LOOPS 20
#define SLEEP_EVERY_N 3

int main( ) 
{
    int nStatic1[NUM_LOOPS], 
        nStaticN[NUM_LOOPS];
    int nDynamic1[NUM_LOOPS], 
        nDynamicN[NUM_LOOPS];
    int nGuided[NUM_LOOPS];
    int i;
    omp_set_num_threads(NUM_THREADS);

    #pragma omp parallel
    {
        #pragma omp for schedule(static, 1)
        for ( i = 0 ; i < NUM_LOOPS ; ++i) 
        {
            if ((i % SLEEP_EVERY_N) == 0) 
                sleep(0);
            nStatic1[i] = omp_get_thread_num( );
        }

        #pragma omp for schedule(static, STATIC_CHUNK)
        for ( i = 0 ; i < NUM_LOOPS ; ++i) 
        {
            if ((i % SLEEP_EVERY_N) == 0) 
                sleep(0);
            nStaticN[i] = omp_get_thread_num( );
        }

        #pragma omp for schedule(dynamic, 1)
        for ( i = 0 ; i < NUM_LOOPS ; ++i) 
        {
            if ((i % SLEEP_EVERY_N) == 0) 
                sleep(0);
            nDynamic1[i] = omp_get_thread_num( );
        }

        #pragma omp for schedule(dynamic, DYNAMIC_CHUNK)
        for ( i = 0 ; i < NUM_LOOPS ; ++i) 
        {
            if ((i % SLEEP_EVERY_N) == 0) 
                sleep(0);
            nDynamicN[i] = omp_get_thread_num( );
        }

        #pragma omp for schedule(guided)
        for ( i = 0 ; i < NUM_LOOPS ; ++i) 
        {
            if ((i % SLEEP_EVERY_N) == 0) 
                sleep(0);
            nGuided[i] = omp_get_thread_num( );
        }
    }

    printf("------------------------------------------------\n");
    printf("| static | static | dynamic | dynamic | guided |\n");
    printf("|    1   |    %d   |    1    |    %d    |        |\n",
             STATIC_CHUNK, DYNAMIC_CHUNK);
    printf("------------------------------------------------\n");

    for ( i=0; i<NUM_LOOPS; ++i) 
    {
        printf("|    %d   |    %d   |    %d    |    %d    |"
                 "    %d   |\n",
                 nStatic1[i], nStaticN[i],
                 nDynamic1[i], nDynamicN[i], nGuided[i]);
    }

    printf("------------------------------------------------\n");
}

------------------------------------------------
| static | static | dynamic | dynamic | guided |
|    1   |    5   |    1    |    5    |        |
------------------------------------------------
|    0   |    0   |    3    |    2    |    0   |
|    1   |    0   |    2    |    2    |    0   |
|    2   |    0   |    2    |    2    |    0   |
|    3   |    0   |    2    |    2    |    0   |
|    0   |    0   |    0    |    2    |    0   |
|    1   |    1   |    0    |    1    |    1   |
|    2   |    1   |    0    |    1    |    1   |
|    3   |    1   |    1    |    1    |    1   |
|    0   |    1   |    1    |    1    |    1   |
|    1   |    1   |    1    |    1    |    3   |
|    2   |    2   |    3    |    0    |    3   |
|    3   |    2   |    3    |    0    |    3   |
|    0   |    2   |    3    |    0    |    2   |
|    1   |    2   |    2    |    0    |    2   |
|    2   |    2   |    2    |    0    |    1   |
|    3   |    3   |    2    |    3    |    1   |
|    0   |    3   | 

## atomic


In [27]:
//%cflags:-fopenmp -lm -D_DEFAULT_SOURCE

//programa para mostrar sentencia nowait y atomic
//mas de 1 hilo tarda mas debido al atomic, dado que no deja que se ejecuten al tiempo
//solucion, partir el trabajo de forma manual

#include <omp.h>
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

#define SIZE 10e06

void nowait_example(long n, long m, double *a, double *b)
{ int i, id;

    #pragma omp parallel //private(id)
    {
        //id = omp_get_thread_num();
        #pragma omp for// nowait
        for (i=1; i<n; i++){
            #pragma omp atomic	//es necesario proteger la acumulación
                *a = *a + (double)i;
        }

        #pragma omp for //nowait
        for (i=0; i<m; i++){
            #pragma omp atomic
                *b = *b + (double)i;
        }
    }
}

int main()
{
    double *a, *b;
    a = malloc(sizeof(double));
    b = malloc(sizeof(double));

    omp_set_num_threads(8);
    *a = 0.0;
    *b = 0.0;
    nowait_example(SIZE, SIZE, a, b);
    printf("\n Total a: %f  b: %f...", *a, *b); fflush(stdout);

    free(a);
    free(b);
}



 Total a: 49999995000000.000000  b: 49999995000000.000000...